In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

2024-03-17 10:43:49.576021: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 10:43:49.576125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 10:43:49.762944: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
dataset_train = pd.read_csv('/kaggle/input/predictive/PS-2-EngineFaultDetection/train_FD001.txt', sep = ' ', header = None).drop([26,27], axis = 1)
col_names = ['id','cycle','setting1','setting2','setting3','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']
dataset_train.columns = col_names
print('Shape of Train Dataset: ', dataset_train.shape)
dataset_train.head()

Shape of Train Dataset:  (20631, 26)


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [3]:
dataset_test = pd.read_csv('/kaggle/input/predictive/PS-2-EngineFaultDetection/test_FD001.txt', sep = ' ', header = None).drop([26,27], axis = 1)
dataset_test.columns = col_names
print('Shape of Test Dataset: ', dataset_test.shape)
dataset_test.head()

Shape of Test Dataset:  (13096, 26)


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [4]:
RUL_FD001=pd.read_csv('/kaggle/input/predictive/PS-2-EngineFaultDetection/RUL_FD001.txt', sep = ' ', header = None).drop([1], axis = 1)
RUL_FD001.columns = ['more']
RUL_FD001['id']=RUL_FD001.index+1
RUL_FD001.head()

,more,id
0,112,1
1,98,2
2,69,3
3,82,4
4,91,5


In [5]:
RUL = pd.DataFrame(dataset_test.groupby('id')['cycle'].max()).reset_index()
RUL.columns = ['id', 'max']
RUL.head()

,id,max
0,1,31
1,2,49
2,3,126
3,4,106
4,5,98


In [6]:
RUL_FD001['rtf'] = RUL_FD001['more']+RUL['max']
RUL_FD001.head()

,more,id,rtf
0,112,1,143
1,98,2,147
2,69,3,195
3,82,4,188
4,91,5,189


In [7]:
print(dataset_train.columns)

Index(['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
       's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
       's15', 's16', 's17', 's18', 's19', 's20', 's21'],
      dtype='object')


In [8]:
y_train = dataset_train["cycle"]

In [9]:
features_col_names = dataset_train.columns[:-1]
X_train = dataset_train[features_col_names]
y_train = dataset_train["cycle"]

In [10]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(dataset_test[features_col_names])

In [11]:
def prepare_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
    return np.array(sequences)

In [12]:
import numpy as np

In [13]:
sequence_length = 50
X_train_lstm = prepare_sequences(X_train_scaled, sequence_length)
y_train_lstm = y_train.values[sequence_length:]

In [14]:
from tensorflow.keras.layers import Input

In [15]:
input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])

In [16]:
model = Sequential([
    Input(shape=input_shape),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=1)
])

In [17]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [18]:
model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=32)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 13395.8994
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 8591.5117
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 6249.7349
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 5316.5532
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 4930.9653
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4879.1030
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4822.0288
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4814.4160
Epoch 9/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4796.3633
Epoch 10/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4765.9927
Epoch 11/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4824.5264
Epoch 12/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4773.2803
Epoch 13/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4867.3472
Epoch 14/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4786.2007
Epoch 15/50


In [19]:
X_test_lstm = prepare_sequences(X_test_scaled, sequence_length)
predictions = model.predict(X_test_lstm)

408/408 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


In [20]:
import matplotlib.pyplot as plt

In [21]:
X_test_scaled = scaler.transform(dataset_test[features_col_names])

In [22]:
X_test = prepare_sequences(X_test_scaled, sequence_length)
y_test = dataset_test["cycle"].values[sequence_length:]

In [27]:
actual_rul = dataset_test["cycle"].values

In [28]:
predictions = model.predict(X_test_lstm)

408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [30]:
X_test_scaled = scaler.transform(dataset_test[features_col_names])
X_test_lstm = prepare_sequences(X_test_scaled, sequence_length)
actual_rul = dataset_test["cycle"].values[sequence_length:]

In [31]:
predictions = model.predict(X_test_lstm)

408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [32]:
actual_rul = actual_rul[:len(predictions)]

In [33]:
mae = mean_absolute_error(actual_rul, predictions)

In [34]:
mse = mean_squared_error(actual_rul, predictions)

In [35]:
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

Mean Absolute Error: 11.16946537981146
Mean Squared Error: 465.18943846393614


In [38]:
rmse = np.sqrt(mse)

In [39]:
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 21.568250704772886
